In [8]:
import tkinter as tk
from tkinter import Scrollbar, Text, filedialog, messagebox
import numpy as np
import tensorflow as tf
import time
from datetime import datetime
import sqlite3
import spacy
import subprocess
import sys
import threading
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tkinter import ttk
import cv2
import speech_recognition as sr
from deepface import DeepFace
from PIL import Image, ImageTk
import pyaudio
import pyttsx3
import requests

# Cargar el modelo desde el archivo .h5
model = load_model('mejor_modelo.h5')

API_URL = 'https://magicloops.dev/api/loop/22dfbc51-f14c-4332-a9c5-4e6e7ebc5305/run'

# ==================================================
# Configuración del Chatbot
# ==================================================

class ChatbotApp:
    def __init__(self, window):
        self.window = window
        # Frame del chatbot
        self.chatbot_frame = ttk.LabelFrame(window, text="Chatbot de Alzheimer")
        self.chatbot_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # Scrollbar
        self.scrollbar = Scrollbar(self.chatbot_frame)
        self.chatbox = Text(self.chatbot_frame, height=15, width=60, yscrollcommand=self.scrollbar.set)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.chatbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Campo de entrada y botón de envío
        self.entry = tk.Entry(self.chatbot_frame, width=50)
        self.entry.pack(pady=5)
        self.send_button = tk.Button(self.chatbot_frame, text="Enviar", command=self.send_message)
        self.send_button.pack()

        # Enviar mensaje al presionar Enter
        self.entry.bind("<Return>", lambda event: self.send_message())

    def send_message(self):
        user_text = self.entry.get().strip()
        if user_text:
            # Mostrar la pregunta en el chat
            self.chatbox.insert(tk.END, f"Tú: {user_text}\n")

            # Enviar la pregunta a la API
            response_text = self.get_response_from_api(user_text)

            # Mostrar la respuesta en el chat
            self.chatbox.insert(tk.END, f"Bot: {response_text}\n\n")
            self.chatbox.see(tk.END)  # Desplazar al final del chat
            self.entry.delete(0, tk.END)  # Limpiar el campo de entrada

    def get_response_from_api(self, question):
        try:
            payload = {"question": question}
            response = requests.get(API_URL, json=payload)

            # Verificar si la respuesta es un JSON válido
            try:
                response_json = response.json()  # Intenta convertir la respuesta a JSON
                if isinstance(response_json, dict):  # Si es un diccionario
                    return response_json.get("response", "Lo siento, no pude obtener una respuesta.")
                elif isinstance(response_json, list) and len(response_json) > 0:  # Si es una lista
                    return response_json[0].get("response", "Lo siento, no pude obtener una respuesta.")
                else:
                    # Si la respuesta no es un diccionario ni una lista, devolver el texto directamente
                    return str(response_json)
            except ValueError:  # Si no se puede convertir a JSON
                # Si la respuesta no es un JSON válido, devolver el texto directamente
                return response.text

        except requests.RequestException as e:
            return f"Error: No se pudo conectar con el servidor. Detalles: {str(e)}"

# ==================================================
# Configuración del Monitoreo
# ==================================================

# Lista de palabras esperadas (puedes modificarla según el contexto)
palabras_esperadas = {"hola", "adiós", "gracias", "por", "favor", "ayuda", "memoria", "alzheimer"}

def detectar_retraso_en_habla(tiempo_transcurrido):
    # Umbral de tiempo para considerar un retraso (3 segundos)
    if tiempo_transcurrido > 3.0:
        return "Alerta: Posible retraso en el habla detectado."
    return None

def detectar_palabras_incorrectas(texto):
    palabras = texto.split()
    palabras_incorrectas = [palabra for palabra in palabras if palabra.lower() not in palabras_esperadas]
    
    if palabras_incorrectas:
        return f"Alerta: Palabras incorrectas detectadas: {', '.join(palabras_incorrectas)}"
    return None

def analizar_fluidez(texto):
    palabras = texto.split()
    if len(palabras) < 3:  # Frase muy corta
        return "Alerta: Frase demasiado corta. Posible dificultad para hablar."

    # Detectar repeticiones
    repeticiones = {}
    for palabra in palabras:
        repeticiones[palabra] = repeticiones.get(palabra, 0) + 1

    for palabra, count in repeticiones.items():
        if count > 3:  # Palabra repetida más de 3 veces
            return f"Alerta: Palabra '{palabra}' repetida {count} veces. Posible dificultad para hablar."
    return None


cap = None
camara_encendida = False

class EnhancedCameraApp:
    def __init__(self, window, window_label, alertas_text):
        self.window = window
        self.window_label = window_label
        self.alertas_text = alertas_text
        self.video_source = 0
        self.vid = None
        self.is_camera_on = False
        self.audio = pyaudio.PyAudio()
        self.muted = False
        self.tts_engine = pyttsx3.init()
        self.voices = self.tts_engine.getProperty('voices')
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.is_listening = False
        self.setup_ui()

        # Vincular el cierre de la ventana para detener el hilo
        self.window.protocol("WM_DELETE_WINDOW", self.on_close)

    def on_close(self):
        # Detener el reconocimiento de voz y la cámara al cerrar la ventana
        self.stop_listening()
        self.stop_camera()
        self.window.destroy()

    def stop_listening(self):
        if self.is_listening:
            self.is_listening = False
            self.btn_listen.config(text="Iniciar Reconocimiento")

    def setup_ui(self):
        control_frame = ttk.Frame(self.window)
        control_frame.pack(pady=10)

        self.btn_frame = ttk.Frame(self.window)
        self.btn_frame.pack(pady=10)

        self.btn_start = ttk.Button(self.btn_frame, text="Iniciar Cámara", command=self.start_camera)
        self.btn_start.pack(side=tk.LEFT, padx=5)

        self.btn_stop = ttk.Button(self.btn_frame, text="Detener Cámara", command=self.stop_camera, state=tk.DISABLED)
        self.btn_stop.pack(side=tk.LEFT, padx=5)

        self.btn_mute = ttk.Button(self.btn_frame, text="Silenciar", command=self.toggle_mute)
        self.btn_mute.pack(side=tk.LEFT, padx=5)

        self.btn_listen = ttk.Button(self.btn_frame, text="Iniciar Reconocimiento", command=self.toggle_listen)
        self.btn_listen.pack(side=tk.LEFT, padx=5)

        self.canvas = tk.Canvas(self.window, width=640, height=480)
        self.canvas.pack()

    def start_camera(self):
        global cap, camara_encendida, monitoreo_activo  # Acceder a las variables globales
        if not self.is_camera_on:
            self.vid = cv2.VideoCapture(self.video_source)
            if not self.vid.isOpened():
                print("Error: No se pudo abrir la cámara.")
                return
            self.is_camera_on = True
            cap = self.vid  # Actualizar la variable global `cap`
            camara_encendida = True  # Actualizar el estado de la cámara
            activar_monitoreo(True)  # Activar el monitoreo
            self.update_camera()
            self.btn_start.config(state=tk.DISABLED)
            self.btn_stop.config(state=tk.NORMAL)

# Modificar la función stop_camera para desactivar el monitoreo
    def stop_camera(self):
        global cap, camara_encendida, monitoreo_activo  # Acceder a las variables globales
        if self.is_camera_on:
            self.is_camera_on = False
            self.vid.release()
            cap = None  # Limpiar la variable global `cap`
            camara_encendida = False  # Actualizar el estado de la cámara
            activar_monitoreo(False)  # Desactivar el monitoreo
            self.canvas.delete("all")
            self.btn_start.config(state=tk.NORMAL)
            self.btn_stop.config(state=tk.DISABLED)

    def toggle_mute(self):
        self.muted = not self.muted
        self.btn_mute.config(text="Activar sonido" if self.muted else "Silenciar")

    def update_camera(self):
        if self.is_camera_on:
            ret, frame = self.vid.read()
            if ret:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(rgb)
                imgtk = ImageTk.PhotoImage(image=img)
                self.canvas.imgtk = imgtk
                self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
            self.window.after(30, self.update_camera)

    def toggle_listen(self):
        global monitoreo_activo
        if not self.is_listening:
            self.is_listening = True
            activar_monitoreo(True)  # Activar el monitoreo
            self.btn_listen.config(text="Detener Reconocimiento")
            threading.Thread(target=self.recognize_speech, daemon=True).start()
        else:
            self.is_listening = False
            activar_monitoreo(False)  # Desactivar el monitoreo
            self.btn_listen.config(text="Iniciar Reconocimiento")




    def recognize_speech(self):
        with self.microphone as source:
        # Ajustar el umbral de ruido ambiental
            self.recognizer.adjust_for_ambient_noise(source, duration=2)  # Ajustar durante 2 segundos
            self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Sistema listo para escuchar...\n")
            self.window.after(0, self.alertas_text.see, tk.END)  # Desplazar al final del texto

            while self.is_listening:
                try:
                    inicio = time.time()  # Tiempo inicial
                    self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Escuchando...\n")
                    self.window.after(0, self.alertas_text.see, tk.END)  # Feedback visual
                    audio = self.recognizer.listen(source, timeout=5)  # Escuchar durante 5 segundos
                    fin = time.time()  # Tiempo final
                    tiempo_transcurrido = fin - inicio

                    texto = self.recognizer.recognize_google(audio, language="es-ES")
                    self.window.after(0, self.display_recognized_text, texto)

                # Detectar retraso en el habla
                    alerta_retraso = detectar_retraso_en_habla(tiempo_transcurrido)
                    if alerta_retraso:
                        self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] {alerta_retraso}\n")
                        self.window.after(0, self.alertas_text.see, tk.END)

                # Detectar palabras incorrectas
                    alerta_palabras_incorrectas = detectar_palabras_incorrectas(texto)
                    if alerta_palabras_incorrectas:
                        self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] {alerta_palabras_incorrectas}\n")
                        self.window.after(0, self.alertas_text.see, tk.END)

                # Analizar fluidez
                    alerta_fluidez = analizar_fluidez(texto)
                    if alerta_fluidez:
                        self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] {alerta_fluidez}\n")
                        self.window.after(0, self.alertas_text.see, tk.END)

                except sr.UnknownValueError:
                    self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] No se pudo entender el audio. Por favor, repite.\n")
                    self.window.after(0, self.alertas_text.see, tk.END)  # Manejo de errores
                except sr.RequestError as e:
                    self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Error en la solicitud al servicio de reconocimiento: {e}\n")
                    self.window.after(0, self.alertas_text.see, tk.END)  # Manejo de errores
                except sr.WaitTimeoutError:
                    self.window.after(0, self.alertas_text.insert, tk.END, f"[{datetime.now()}] Tiempo de espera agotado. Intenta de nuevo.\n")
                    self.window.after(0, self.alertas_text.see, tk.END)  # Manejo de errores

    def display_recognized_text(self, text):
        self.alertas_text.insert(tk.END, f"[Voz Detectada] {text}\n")
        self.alertas_text.see(tk.END)

# Función para detectar actividad física
prev_frame = None
def detectar_movimiento():
    global prev_frame, cap
    if not camara_encendida or cap is None:
        return False
    ret, frame = cap.read()
    if not ret:
        return False
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    if prev_frame is None:
        prev_frame = gray
        return False
    frame_delta = cv2.absdiff(prev_frame, gray)
    prev_frame = gray
    movimiento = np.sum(frame_delta) > 500000  # Umbral de cambio en píxeles
    return movimiento

# Función para detectar emociones
emociones_traduccion = {
    "angry": "Enojo",
    "disgust": "Disgusto",
    "fear": "Miedo",
    "happy": "Feliz",
    "sad": "Tristeza",
    "surprise": "Sorpresa",
    "neutral": "Neutral"
}

def detectar_emocion():
    global cap
    if not camara_encendida or cap is None:
        return "No se detecta cámara"
    ret, frame = cap.read()
    if not ret:
        return "No se pudo capturar imagen"
    try:
        # Detectar rostros en la imagen usando DeepFace.extract_faces
        rostros = DeepFace.extract_faces(frame, detector_backend='mtcnn')

        if len(rostros) > 0:
            # Tomar el primer rostro detectado
            rostro = rostros[0]['face']
            
            # Analizar la emoción en el rostro detectado
            resultados = DeepFace.analyze(rostro, actions=['emotion'], enforce_detection=False)
            emocion = resultados[0]['dominant_emotion']
            
            # Traducir la emoción al español
            emocion_espanol = emociones_traduccion.get(emocion, "Emoción desconocida")
            return f"Emoción detectada: {emocion_espanol}"
        else:
            return "No se detectó un rostro en la imagen"
    except Exception as e:
        return f"Error al detectar emoción, es posible que la cámara no esté detectando correctamente al usuario, por favor, asegurese de que la cámara capte totalmente el rostro de la persona"

# Configuración de reconocimiento de voz
recognizer = sr.Recognizer()
def detectar_cambios_habla():
    try:
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source)
            try:
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
                texto = recognizer.recognize_google(audio, language="es-ES")
                return texto
            except sr.WaitTimeoutError:
                return "Alerta: No se detectó sonido en el tiempo esperado."
            except sr.UnknownValueError:
                return "Alerta: No se detectó habla."
            except sr.RequestError:
                return "Alerta: Error en reconocimiento."
    except OSError:
        return "Alerta: No se detecta micrófono, no se puede realizar monitoreo del habla."

# Variable global para controlar el estado del monitoreo
monitoreo_activo = False
app_running = True

# Función para activar/desactivar el monitoreo
def activar_monitoreo(estado):
    global monitoreo_activo
    monitoreo_activo = estado
    

# Monitoreo en tiempo real
def monitorear_paciente():
    global app_running
    while app_running:  # Solo monitorear si la aplicación está en ejecución
        if monitoreo_activo:
            actividad_fisica = detectar_movimiento()
            habla = detectar_cambios_habla()
            emocion = detectar_emocion()
            alertas = []
            
            if not actividad_fisica:
                alertas.append("Alerta: Baja actividad física detectada.")
            if "No se detectó habla" in habla:
                alertas.append("Alerta: Cambios en el habla detectados.")
            
            alertas.append(emocion)
            
            for alerta in alertas:
                alertas_text.insert(tk.END, f"[{datetime.now()}] {alerta}\n")
                alertas_text.see(tk.END)
        
        time.sleep(8)

def on_close():
    global app_running
    app_running = False  # Detener el hilo de monitoreo
    camera_app.on_close()  # Detener el reconocimiento de voz y la cámara
    ventana.destroy()



# Función para predecir
def seleccionar_imagen():
    file_path = filedialog.askopenfilename()
    title="Selecciona una imagen",
    filetypes=(("Archivos de imagen", ".jpg *.jpeg *.png"), ("Todos los archivos", ".*"))
    return file_path

def predecir_imagen(file_path):
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=(150, 150))
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediccion = model.predict(img)
    clase_predicha = np.argmax(prediccion, axis=1)
    return clase_predicha[0]

def predecir_y_mostrar():
    file_path = seleccionar_imagen()
    if file_path:
        clase_predicha = predecir_imagen(file_path)
        if clase_predicha == 0:
            resultado = "La imagen proporcionada pertenece a una persona que se encuentra en la etapa leve del alzheimer.\n Se sugiere que opte por participar en actividades cognitivas y físicas para estimular el cerebro.\n Consultar regularmente in médico especializado en neurología.\n  Se observa una disminución en el volumen del tejido cerebral, especialmente en áreas como el hipocampo (crucial para la memoria) y la corteza cerebral. Esta atrofia es un indicador clave de la enfermedad de Alzheimer"
        elif clase_predicha == 1:
            resultado = "La imagen proporcionada pertenece a una persona que se encuentra en la etapa avanzada del alzheimer. \n Se observa una atrofia cerebral más pronunciada. \n Los ventrículos están más agrandados y la corteza cerebral muestra una mayor pérdida de volumen.\n Se recomienda evaluaciones médicas de forma frecuente"
        elif clase_predicha == 2:
            resultado = "La imagen proporcionada pertenece a una persona que no posee alzheimer. \n  En general, la estructura cerebral parece estar dentro de los límites normales para la edad de la persona. \n No se observa una atrofia cerebral significativa. El volumen del tejido cerebral parece conservado."
        elif clase_predicha == 3:
            resultado = "La imagen proporcionada pertenece a una persona que puede tener indicios de padecer alzheimer. \n Se observa que el patrón de atrofia es consistente con la enfermedad de Alzheimer en una etapa muy temprana. Por lo tanto, existe una posibilidad de que esta persona padezca de alzheimer "
        else:
            resultado = "La imagen proporcionada es incorrecta."
        resultado_text.delete(1.0, tk.END)  # Limpiar el texto anterior
        resultado_text.insert(tk.END, resultado)

# ==================================================
# Interfaz gráfica
# ==================================================

ventana = tk.Tk()
ventana.title("Memory Vision")
ventana.geometry("800x600")

notebook = ttk.Notebook(ventana)
notebook.pack(fill=tk.BOTH, expand=True)

### Pestaña Monitoreo ###
monitoreo_frame = ttk.Frame(notebook)
notebook.add(monitoreo_frame, text="Monitoreo")

alertas_frame = tk.LabelFrame(monitoreo_frame, text="Alertas")
alertas_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

alertas_text = tk.Text(alertas_frame, height=15, width=80)
alertas_text.pack(fill=tk.BOTH, expand=True)

camara_frame = ttk.Frame(monitoreo_frame)
camara_frame.pack(pady=10)

camara_label = tk.Label(camara_frame)
camara_label.pack()

camera_app = EnhancedCameraApp(ventana, camara_label, alertas_text)

boton_camara = tk.Button(monitoreo_frame, text="Encender Cámara", command=camera_app.start_camera)
boton_camara.pack()

### Pestaña Chatbot ###
chatbot_frame = ttk.Frame(notebook)
notebook.add(chatbot_frame, text="Chatbot")

chatbot_system = ChatbotApp(chatbot_frame)

### Pestaña Predicción ###
prediccion_frame = ttk.Frame(notebook)
notebook.add(prediccion_frame, text="Predicción")

pred_frame = tk.LabelFrame(prediccion_frame, text="Predicción de la imagen")
pred_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

predecir_button = tk.Button(pred_frame, text="Seleccionar Imagen y Predecir", command=predecir_y_mostrar)
predecir_button.pack(pady=5)

resultado_text = tk.Text(pred_frame, height=5, width=80)
resultado_text.pack(fill=tk.BOTH, expand=True)

# Iniciar monitoreo en segundo plano
monitoreo_thread = threading.Thread(target=monitorear_paciente, daemon=True)
monitoreo_thread.start()

ventana.mainloop()
ventana.protocol("WM_DELETE_WINDOW", on_close)

: 